[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_ACF_PACF_identification/EMQA_ACF_PACF_identification.ipynb)

# EMQA_ACF_PACF_identification

**ACF / PACF Pattern Identification Guide**

A 4x2 grid showing the theoretical ACF and PACF signatures of White Noise, AR(1), MA(1), and ARMA(1,1).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf

np.random.seed(42)
n = 500
nlags = 25

configs = [
    {'ar': [1], 'ma': [1], 'label': 'White Noise',
     'acf_note': 'All lags near zero', 'pacf_note': 'All lags near zero'},
    {'ar': [1, -0.9], 'ma': [1], 'label': 'AR(1)',
     'acf_note': 'Exponential decay', 'pacf_note': 'Cuts off after lag 1'},
    {'ar': [1], 'ma': [1, 0.8], 'label': 'MA(1)',
     'acf_note': 'Cuts off after lag 1', 'pacf_note': 'Exponential decay'},
    {'ar': [1, -0.7], 'ma': [1, 0.5], 'label': 'ARMA(1,1)',
     'acf_note': 'Tails off (mixed decay)', 'pacf_note': 'Tails off (mixed decay)'},
]

fig, axes = plt.subplots(4, 2, figsize=(14, 14))
conf_bound = 1.96 / np.sqrt(n)

for row, cfg in enumerate(configs):
    proc = ArmaProcess(np.array(cfg['ar']), np.array(cfg['ma']))
    y = proc.generate_sample(nsample=n)

    acf_vals = acf(y, nlags=nlags)
    pacf_vals = pacf(y, nlags=nlags)

    # ACF
    ax_acf = axes[row, 0]
    ax_acf.bar(range(nlags + 1), acf_vals, width=0.3, color=COLORS['blue'], alpha=0.85)
    ax_acf.axhline(conf_bound, color=COLORS['red'], linestyle='--', linewidth=0.8)
    ax_acf.axhline(-conf_bound, color=COLORS['red'], linestyle='--', linewidth=0.8)
    ax_acf.axhline(0, color='black', linewidth=0.5)
    ax_acf.set_title(f'{cfg["label"]} — ACF', fontsize=11, fontweight='bold')
    ax_acf.set_xlabel('Lag')
    ax_acf.set_ylabel('ACF')
    props = dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.8)
    ax_acf.text(0.98, 0.95, cfg['acf_note'], transform=ax_acf.transAxes,
                fontsize=9, ha='right', va='top', bbox=props)

    # PACF
    ax_pacf = axes[row, 1]
    ax_pacf.bar(range(nlags + 1), pacf_vals, width=0.3, color=COLORS['green'], alpha=0.85)
    ax_pacf.axhline(conf_bound, color=COLORS['red'], linestyle='--', linewidth=0.8)
    ax_pacf.axhline(-conf_bound, color=COLORS['red'], linestyle='--', linewidth=0.8)
    ax_pacf.axhline(0, color='black', linewidth=0.5)
    ax_pacf.set_title(f'{cfg["label"]} — PACF', fontsize=11, fontweight='bold')
    ax_pacf.set_xlabel('Lag')
    ax_pacf.set_ylabel('PACF')
    ax_pacf.text(0.98, 0.95, cfg['pacf_note'], transform=ax_pacf.transAxes,
                 fontsize=9, ha='right', va='top', bbox=props)

fig.suptitle('ACF / PACF Identification Guide', fontsize=14, fontweight='bold', y=1.01)
fig.tight_layout()
save_fig(fig, 'arma_acf_pacf_identification.pdf')
plt.show()
